## Трек «CV-инженер» 
## Проект 1. Угадай знаменитость 
#### Итоговая работа Анны Лубневской

In [ ]:
#Imports
import torch, os
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
#LoadData
import yaml
import cv2
#Show dataset loading visualization
#from tqdm import tqdm

#---------------------------------------------------------------------------------------------

#Serializer
def deserialize(file_name):
    with open(file_name, 'r') as f:
        obj =  yaml.load(f, Loader=yaml.FullLoader)
        return (obj['img_file_name'], obj['data'])
        
#Defining dataset class
class Faces_dataset(Dataset):
    def __init__(self,transform=None):
        self.data =[]#data of cv2 pics and labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img=self.data[idx][0]
        label=self.data[idx][1]
        if self.transform:
            img=self.transform(img)
        return (img, label)
    
    def add(self, item, label):
        self.data.append((item,label))

    #Create dataset function
def load_data(location,classid,dataset=None):
    if dataset is None:
        dataset=Faces_dataset(ToTensor())
    mainPic=None
    roiList=[]
    roi_size=(64,64)
    for yml in os.listdir(location):
        if yml.endswith(".yaml"):
            fname,roiList=deserialize(os.path.join(location, yml))
                #ROI
            if os.path.isfile(os.path.join(location, fname)):
                mainPic=cv2.imread(os.path.join(location, fname))
                for p1, p2 in roiList:
                    p1 = tuple([int(v) for v in p1])
                    p2 = tuple([int(v) for v in p2])
                    roi=mainPic[p1[1]:p2[1],p1[0]:p2[0]]
                    roi=cv2.resize(roi,roi_size)
                    dataset.add(roi,classid)
            mainPic=None   
    return dataset
    
#---------------------------------------------------------------------------------------------    

#Defining model class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 2, 4)
        self.conv2 = nn.Conv2d(2, 4, 4)
        self.conv3 = nn.Conv2d(4, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6*5*5, 35)
        self.fc2 = nn.Linear(35, 8)
        self.fc3 = nn.Linear(8, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,6*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#Train function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#Test function
best_pr=0
current_pr=0
def test(dataloader, model,loss_fn):
    global current_pr
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    current_pr=100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
#---------------------------------------------------------------------------------------------

#Defining device (cpu/gpu).
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

#Helpers
BASE_DIR=os.getcwd()
MODEL_DIR=os.path.join(BASE_DIR,'model')
#Model
model = NeuralNetwork().to(device)
USE_SAVED_MODEL=True
LOAD_PATH=os.path.join(MODEL_DIR,"model.pth")
SAVE_TRAINED_MODEL=False
SAVE_PATH=os.path.join(MODEL_DIR,"model.pth")
#Params
LOAD_DATA_TRAIN=True
LOCATION_DATA_TRAIN="data/train"
LOCATION_BAD_DATA_TRAIN="data/badtrain"
LOAD_DATA_TEST=True
LOCATION_DATA_TEST="data/test"
LOCATION_BAD_DATA_TEST="data/badtest"
#Hyperparams
BATCH_SIZE=32
EPOCHS=10000
LEARNING_RATE=1e-9
MOMENTUM=1
#Loss and optimizer
LOSS_FN = nn.CrossEntropyLoss() #Loss calculation function
OPTIMIZER = torch.optim.SGD(model.parameters(), LEARNING_RATE, MOMENTUM) #Train function

#---------------------------------------------------------------------------------------------

    
#Load model to device and print it
if(USE_SAVED_MODEL):
    model.load_state_dict(torch.load(LOAD_PATH)) #load saved model
print(model)

#Training data
if(LOAD_DATA_TRAIN):
    training_data=load_data(LOCATION_DATA_TRAIN,1)
    training_data=load_data(LOCATION_BAD_DATA_TRAIN,0,training_data)
    print("Training data count:",len(training_data))
#Test data
if(LOAD_DATA_TEST):
    test_data=load_data(LOCATION_DATA_TEST,1)
    test_data=load_data(LOCATION_BAD_DATA_TEST,0,test_data)
    print("Test data count:",len(test_data))


#Data loaders
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=len(test_data))


#Data loaders info (!)
'''
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break
'''
    
    
#Training
first=True
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    if first:
        test(test_dataloader, model,LOSS_FN)
        best_pr=current_pr
    train(train_dataloader, model, LOSS_FN, OPTIMIZER)
    test(test_dataloader, model,LOSS_FN)
    if best_pr<current_pr:
        torch.save(model.state_dict(), SAVE_PATH)
        best_pr=current_pr
print("Done!")

#Save
if(SAVE_TRAINED_MODEL):
    torch.save(model.state_dict(), SAVE_PATH)
    print("Saved PyTorch Model State to \"{0}\"".format(SAVE_PATH))


#Defined classes of outputs
classes = [
    "Not face",
    "Face"
]

#Model in use (!)
'''
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
'''
print("end")

Using cuda device
NeuralNetwork(
  (conv1): Conv2d(3, 2, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(2, 4, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(4, 6, kernel_size=(4, 4), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=150, out_features=35, bias=True)
  (fc2): Linear(in_features=35, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=2, bias=True)
)
Training data count: 403
Test data count: 171
Epoch 1
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Erro

Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 47
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 48
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 49
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 51
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

loss: 0.000925  [    0/  403]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.004537 

Epoch 52
-------------------------------
Test Error: 
 Accuracy: 86.5%, A

**ИТОГОВЫЙ РЕЗУЛЬТАТ, которого удалось добиться:**

Accuracy: 86.5%  
Avg loss: 0.004537